# Wikipedia Analysis

This notebook IS NOT an skeleton. Is a sample of instruccions to analyse in classroom the Wikipedia Dataset provided by Databricks

During the class we will fill cells to implement the PageRank algorithm

# Steps to implement **Page Rank Algorithm**
* Check Data Raw structure
* Extract relevant data: [Document ID, List of links]
* Transform *List of links* to *List od Docuemts ID*: *Forward Links Table*
* Calculate *Number of output links*
* Construct *Reverse Links Table* from *Forward Links Table*
* Initialize *Page Rank Table*
* Recalculate *Page Rank Table* until:
 * All the *Page Rank* values are stable
 * Reach number of iterations (sugested value: 20 iterations)

# A) Library configuration

In [1]:
import pandas as pd
import re
import math 
import warnings
warnings.filterwarnings('ignore')

In [2]:
# from pyspark.sql.types import *
from pyspark.sql.types import ArrayType, StringType,LongType, FloatType
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from operator import add 

In [3]:
spark.conf.set("spark.sql.execution.arrow.enabled", "false")

22/01/03 15:25:10 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


# 1) Check Data Raw Structure
We will uses Databrics Wikipedia dataset, which contains 2012 Wikipedia Database in english

In [5]:
%fs ls dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia

UsageError: Line magic function `%fs` not found.


In [0]:
%fs ls dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet

path,name,size
dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet/_SUCCESS,_SUCCESS,0
dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet/_common_metadata,_common_metadata,779
dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet/_metadata,_metadata,15986429
dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet/part-r-00000-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,part-r-00000-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,10085419
dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet/part-r-00001-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,part-r-00001-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,15346826
dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet/part-r-00002-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,part-r-00002-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,15762593
dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet/part-r-00003-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,part-r-00003-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,8977575
dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet/part-r-00004-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,part-r-00004-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,15739954
dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet/part-r-00005-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,part-r-00005-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,9298525
dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet/part-r-00006-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,part-r-00006-ff5bbd4d-6518-4e17-8ff5-c8fd3d4a5b86.gz.parquet,5532126


Here we defines the wikipediaDF Spark Dataframe, with the full database content:
We need to know the total number of documents in this database.

In [6]:
wikipediaDF=spark.read.parquet("dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet")

Py4JJavaError: An error occurred while calling o30.parquet.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "dbfs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3281)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3301)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:366)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:297)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:286)
	at scala.

In [0]:
wikipediaDF.count()

Out[6]: 5823210

To program the PageRank Algorithm, we need to extract a subset of the full database. We will select 0.00001 of the full database, and to avoid randomness behaviour, we set to a fixed seed value=0.

**Note:** For the final evaluation, should change the fraction to 0.001

In [0]:
PartialWikipediaDF=wikipediaDF.sample(fraction=0.001,seed=0).cache()

In [0]:
PartialWikipediaDF.count()

Out[15]: 13

Now, we can check the data structure:

In [0]:
display(PartialWikipediaDF)

title id revisionId revisionTimestamp revisionUsername revisionUsernameId text Juliet O'Neill 7009414 672322994 2015-07-20T20:31:11.000+0000 Egsan Bacon 603343 '''Juliet O'Neill ''' is a [[Canada|Canadian]] journalist who was the subject of controversy when the [[Royal Canadian Mounted Police]] raided her house in 2004 in an attempt to find the source of an alleged internal leak giving her access to privileged documents related to the [[Maher Arar]] case. ==Career== In 1986, O'Neill was working for the [[Canadian Press]] newswire, when she caught international attention for being the only reporter to capture [[Sondra Gotlieb]] slapping her social [[secretary]] Connie Gibson Connor at a [[state dinner]] held to honour Canadian Prime Minister [[Brian Mulroney]] and US Vice President [[George H. Bush]]. [[Washington Monthly]], [http://findarticles.com/p/articles/mi_m1316/is_n5_v22/ai_9085523 Washington Rollercoaster. - book reviews], 1990 In 2004, she was a reporter with the ''[[Ottawa Citizen]]'' at the time of the raid, and she had earlier served as a foreign correspondent in [[Moscow]]. [http://www.taddlecreekmag.com/alfredfebruary Those Were the Days | Taddle Creek | www.taddlecreekmag.com<!-- Bot generated title -->] ==House raid== On January 21, 2004 the [[Royal Canadian Mounted Police]] raided her house as part of an attempt to identify the leak. During the raid, police seized notebooks, files, hard drives and other materials. In November 2004, [[Ontario Superior Court]] Judge [[Lynn Ratushny]] ruled that the sealing of the search warrants was unacceptable, although [[Justice of the Peace]] [[Richard Sculthorpe]] had given approval after the RCMP invoked the Security of Information Act. Justice Ratushny stated that the sealing of the search violated guarantees of a free press, freedom of expression and the publics right to an open court system. She ordered that a [[Sanitization (classified information)|redact]]ed copy be released to the public. [http://www.injusticebusters.com/04/ONeill_Juliet.htm injusticebusters 2004 > > Juliet O'Neill: Traumatising a reporter<!-- Bot generated title -->] In October 2006, Ontario Superior Court Judge Ratushny struck down Section 4 of the Security of Information Act, {{cite web |url=http://laws.justice.gc.ca/en/O-5/275827.html#rid-275834 |title=Section 4 of the Security of Information Act |date=2006-09-15 |accessdate= 2006-10-26 |format= |work=Consolidated Statutes and Regulations |publisher= Department of Justice Canada}} ruling that it was "unconstitutionally vague" and broad {{cite news | first = Ian | last = MacLeod | authorlink = | author = | coauthors = | title = Decision offers chance to overhaul security act | url = http://www.canada.com/ottawacitizen/news/story.html?id=5f16f753-ce2a-48c3-85d3-b070889403b0&p=1 | format = | work = The Ottawa Citizen | publisher = | pages = | page = | date = October 20, 2006 | accessdate = 2006-10-26 }} and an infringement of freedom of expression. All materials seized from O'Neill were ordered returned. O'Neill was represented by attorneys Rick Dearden and Wendy Wagner from [[Gowlings]] a law firm with offices in Ottawa. {{cite web|url=http://www.gowlings.com/resources/PublicationPDFs/Wagner_Dearden_Nov06.pdf|author=Richard Dearden and Wendy Wagner|title=The New Threat To Access To Information: National Security Allegations|publisher=Gowlings|accessdate=2013-09-21}} ==Awards and honors== In 1996 O'Neill was awarded a [[Southam Fellowship]]. In 2005 she was honored with the [http://www.ccwpf-cclpm.ca/press-freedom-day-lunch/past-winners World Press Freedom Award ]. In 2009 she was honored with the [[Charles Lynch Award]] for Outstanding Coverage of National Affairs. [http://www.ottawacitizen.com/Canwest+editor+wins+national+press+award/2118676/story.html Ottawa Citizen report on 2009 Charles Lynch Award] ==References== {{Reflist}} ==External links== *[http://www.julietoneill.com Juliet O'Neill's website] *[http://www.media-awareness.ca/english/resou

## Conclusions with the data raw analysis:
* There are several columns, but the relevant information is stored in the follow columns:
 * **title**: The title of the document.
 * **id**: Id of the document
 * **text**: The content of the document. The most relevant information here (for the page rank algorithm) are the links to other documents. The link is enclosed in brackets, and contains the title of the document.

# 2) Extract Relevant Data
From the previous conclusions, we knows we need to select just three columns [*title*,*id*,*text*], and the relevant information from the *text* column are the links, identified by the titles enclosed in brackets.

Here we need to use regular expressions to select the relevant information from the *text* column.

Here we will implement a *parse_links* function, who receive a string and return a list of strings with the titles of the pointed documents.

This is a Python function, so is not direct callable from the Spark Dataframe, so, we need to define also the User Defined Function, to be usable in Spark Dataframes.

In [0]:
# This is a suboptimal function. Should be improved, to detect more complex links, links to documents in image references, etc...
# ESTUDIAR ESTE ARTICULO: https://es.wikipedia.org/wiki/Wikipedia:Estructura_de_un_art%C3%ADculo 
def parse_links(document_body):
  document_body_u = document_body.lower() 
  data=re.findall(r'\[\[(?!category|wikipedia|file|help|special)((?:(?![\[\]]).)*)\]\]',document_body_u)
  if (len(data)>0):
    links=[s.split('|')[0].split('#')[0].lower() for s in data]
  else:
    links=[]
  return links

In [0]:
parse_links_udf = udf(parse_links,ArrayType(StringType()))

It is necesarly convert the text to lowercase (both: *title* and *text* columns)

In [0]:
tolower_udf= udf(lambda x: x.lower())

Now, we create parseDF with the selected information, renaming the result tables to "title", and "links"

In [0]:
parsedDF = PartialWikipediaDF.select(tolower_udf("title").alias("title"),"id",parse_links_udf("text").alias("links"))

In [0]:
display(parsedDF)

title,id,links
juliet o'neill,7009414,"List(canada, royal canadian mounted police, maher arar, canadian press, sondra gotlieb, secretary, state dinner, brian mulroney, george h. bush, washington monthly, ottawa citizen, moscow, royal canadian mounted police, ontario superior court, lynn ratushny, justice of the peace, richard sculthorpe, sanitization (classified information), gowlings, southam fellowship, charles lynch award)"
manettia,6379698,"List(plantae, angiosperms, eudicots, asterids, gentianales, rubiaceae, rubioideae, hedyotideae, mutis, linnaeus, genus, liana, herb, rubiaceae, tropical, americas, synonymy, carapichea ipecacuanha, manettia skutchii)"
kenny mckinley,22371574,"List(denver broncos, wide receiver, mableton, georgia, centennial, colorado, south carolina gamecocks football, denver broncos, southeastern conference, american football, wide receiver, denver broncos, national football league, 2009 nfl draft, college football, south carolina gamecocks football, mableton, georgia, austell, georgia, touchdowns, sidney rice, turf toe, south carolina gamecocks football, southeastern conference, south carolina gamecocks football, tennessee volunteers football, all-america, south carolina gamecocks football, hamstring, southeastern conference, denver broncos, 2009 nfl draft, 2009 nfl draft, associated press, yahoo! news, injured reserve, arapahoe county, colorado, depression (mood), associated press, tennessee titans)"
"brown-westhead, moore & co",39249641,"List(meissen porcelain, china, earthenware, sanitary, cauldon canal, stoke-upon-trent, england, victorian majolica)"
phalonidia haplidia,42305636,"List(animal, arthropod, insect, lepidoptera, tortricidae, phalonidia, józef razowski, moth, tortricidae, mexico)"
bean bowl,13511010,"List(scottsbluff, nebraska, doane college, university of northern colorado, national collegiate athletic association, united states, college football, bowl game, scottsbluff, nebraska, idaho state university, chadron state college, new york times, doane college, university of northern colorado, doane college)"
benjamin karamoko,45446916,"List(mali, defender (association football), as saint-étienne, as saint-étienne, as saint-étienne, france, defender (association football), ligue 1, as saint-étienne, as saint-étienne, ligue 1, girondins de bordeaux, paul baysse)"
villa aldama,23255883,"List(municipalities of veracruz, administrative divisions of mexico, veracruz, north american central time zone, north american central time zone, veracruz (state), xalapa, las minas, altotonga (municipality), las vigas de ramírez, perote (municipality), maize, beans, potato, faba bean, virgin of guadalupe)"
hypolycaena merguia,36575375,"List(animal, arthropod, insect, lepidoptera, lycaenidae, hypolycaena, william doherty, butterfly, lycaenidae, south-east asia)"
miss chinese international pageant 2010,29406590,"List(eliza sam, carol cheng, derek li, raymond lam, leo ku, liu xuan (gymnast), tianjin, china, television broadcasts limited, miss chinese international pageant, tianjin, china, tvb, miss chinese international pageant 2009, christine kuo, toronto, canada, eliza sam, vancouver, canada, miss chinese international pageant 1989, carol cheng, raymond lam, leo ku, kuala lumpur, eliza sam, vancouver, kuala lumpur, candy chang, toronto, auckland, foshan, hong kong, amsterdam, melbourne, montréal, new york city, sydney, henan, auckland, harbin, amsterdam, chaozhou, bangkok, wenchang, chicago, taiwan, foshan, shandong, hong kong, shunde, kuala lumpur, fujian, macau, guangdong, manila, fujian, melbourne, hainan, montréal, fujian, new york city, jilin, san francisco, guangxi, seattle, foshan, singapore, fujian, sydney, beijing, tahiti, guangdong, candy chang, toronto, beijing, eliza sam, vancouver, guangzhou, hebei, chaoshan, guangzhou, shenyang, henan, henan, jianxi, shandong, guangdong, gansu, vancouver, eliza sam, sydney)"


# 3) Transform *List of links* to *List od Docuemts ID*: *Forward Links Table*

To get the *id* of the target documents, we need analyse the full Wikipedia Database and extract a table with this two information.

This information is static, and used in distributed way. So, we will collect the data and convert to a Pandas Dataframe (PDF suffix).

In [0]:
titleidDF=wikipediaDF.select("id",tolower_udf("title").alias("title"));

In [0]:
titleidPDF=titleidDF.toPandas()

In [0]:
ForwardDF.printSchema()

root
 |-- id: integer (nullable = true)
 |-- links: array (nullable = true)
 |    |-- element: long (containsNull = true)



In [0]:
titleidPDF

,id,title
0,13758,history of physics
1,13761,hydrofoil
2,13763,henri chopin
3,13764,hassium
4,13768,hydrus
...,...,...
5823205,18224979,koama
5823206,18224984,kokoaga-ouest
5823207,18224986,craig (given name)
5823208,18224990,"kou, burkina faso"


In [0]:
def text_links_2idx(links , title_idxPDF):
  if ( len(links)>0):
  # This command looks in the title column if the elements in the list links exists, and if it exists
  # gets his id value. The result is converted to a list.
    result = title_idxPDF[title_idxPDF.title.isin(links)].id.to_list()
  else:
    result = [] 
  return result

In [0]:
broadcast_title_idPDF = sc.broadcast(titleidPDF)

In [0]:
def text_links_2idx_2(links):
  title_idxPDF = broadcast_title_idPDF.value
  if ( len(links)>0):
  # This command looks in the title column if the elements in the list links exists, and if it exists
  # gets his id value. The result is converted to a list.
    result = title_idxPDF[title_idxPDF.title.isin(links)].id.to_list()
  else:
    result = [] 
  return result

We define a test_list, to check the the text_links_2idx function.

In [0]:
test_list=["shenzhou (spacecraft)|shenzhou","shenzhou (spacecraft)|shenzhou", "long march 2f|chang zheng 2f", "jiuquan satellite launch center|jiuquan", "jiuquan launch area 4|la-4/sls-1", "geocentric orbit|geocentric", "low earth orbit|low earth", "yang liwei", "shenzhou 4", "shenzhou 6", "shenzhou program|shenzhou", "human spaceflight", "chinese space program", "shenzhou spacecraft", "long march 2f", "soviet union", "russia", "united states", "file:yang liwei.jpg|thumb|left|[[yang liwei", "yang liwei", "chinese nationalism", "united nations", "people's daily", "spacedaily", "political status of taiwan|taiwan", "people's daily", "general secretary of the communist party of china|general secretary", "president of the people's republic of china|president", "hu jintao", "great hall of the people", "sina.com", "central committee of the communist party of china|cpc central committee", "state council of china|state council", "central military commission", "sina.com", "prime minister of japan|prime minister", "junichiro koizumi", "cctv.com", "george w. bush", "united states department of state|u.s. state department", "sean o'keefe", "dprk", "kwangmyŏngsŏng-1", "yang liwei", "chinese space program", "tiangong program", "shenzhou spacecraft", "long march rocket", "jiuquan satellite launch center", "shenzhou 4", "shenzhou 6", "category:human spaceflights|shenzhou 05", "category:shenzhou program|shenzhou 05", "category:spacecraft launched in 2003", "category:2003 in china"]


In [0]:
idx_list=text_links_2idx(test_list,titleidPDF)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-1782843817263986> in <module>
----> 1 idx_list=text_links_2idx(test_list,titleidPDF)

NameError: name 'test_list' is not defined

In [0]:
idx_list

Out[167]: [25391,
 26779,
 31769,
 340917,
 340934,
 340947,
 151210,
 153223,
 263163,
 357847,
 553623,
 869878,
 489179,
 2481352,
 3434750,
 4980140,
 3414021,
 22309268]

In [0]:
i877 idx_list

In [0]:
udf_text_links_2idx =udf(text_links_2idx_2,ArrayType(LongType()))

In [0]:
ForwardDF = parsedDF.select("id",udf_text_links_2idx("links").alias("links")).cache()

In [0]:
display(ForwardDF)

id,links
7009414,"List(69080, 63403, 325569, 285314, 1168864, 5042916, 6143251, 8455935, 1313445)"
6379698,"List(29833, 38493, 551448, 2799672, 2774053, 12895533, 24542769, 43699347, 28873728, 43699347, 54025, 54080)"
22371574,"List(21211, 30839, 6771, 8122, 735314, 606746, 97016, 1560005, 109983, 109986, 7357018, 6845548, 18951490, 17144160, 12931098)"
39249641,"List(5405, 344775, 173301, 12080970, 9316)"
42305636,"List(53307, 66633, 638719, 3966054, 11039790, 25900414, 19827221, 41257955, 41257955, 23366462)"
13511010,"List(6771, 60706, 402032, 362136, 338634, 124204, 3434750, 5104139)"
45446916,"List(19127, 5843419, 17566417)"
23255883,"List(23501, 63125, 154564, 325388, 87990, 16086176, 20656228, 22901605)"
36575375,"List(48338, 53307, 881635, 9500275, 11039790, 19827221, 14624492, 23366462)"
29406590,"List(32706, 25734, 27318, 27862, 12537, 13404, 5405, 49728, 50821, 56615, 64646, 166411, 343968, 645042, 180264, 143373, 1029003, 2231866, 1461079, 2146364, 1827332, 11388236, 22821281, 18603746, 17306237, 21157709, 21044297, 47836950, 18660332, 36902294, 34743894, 34743894)"


In [0]:
ForwardDF.show()

+--------+--------------------+
|      id|               links|
+--------+--------------------+
| 7009414|[69080, 63403, 32...|
| 6379698|[38493, 551448, 2...|
|22371574|[21211, 30839, 67...|
|39249641|[5405, 344775, 17...|
|42305636|[53307, 66633, 63...|
|13511010|[6771, 124204, 51...|
|45446916|   [19127, 17566417]|
|23255883|[23501, 63125, 15...|
|36575375|[48338, 53307, 88...|
|29406590|[32706, 25734, 27...|
|41834571|[14653, 448049, 7...|
|46653940|[5489, 29032274, ...|
|15522609|[12108, 39696, 46...|
+--------+--------------------+



Out[97]: [Row(locate(7009414, id, 0)=0),
 Row(locate(7009414, id, 0)=0),
 Row(locate(7009414, id, 0)=0),
 Row(locate(7009414, id, 0)=0),
 Row(locate(7009414, id, 0)=0),
 Row(locate(7009414, id, 0)=0),
 Row(locate(7009414, id, 0)=0),
 Row(locate(7009414, id, 0)=0),
 Row(locate(7009414, id, 0)=0),
 Row(locate(7009414, id, 0)=0),
 Row(locate(7009414, id, 0)=0),
 Row(locate(7009414, id, 0)=0),
 Row(locate(7009414, id, 0)=0)]

Out[102]: [Row(s=2)]

Once verified the function, we need to define the UDF function, to invoke it from the **parsedDF** to select just [id, list of ids].

To be efficient, we can broadcast the variable *titleidPDF*, to call it in the transformation.

We will call this Dataframe as: **ForwardDF**

# 4) Calculate *Number of output links*
Using the **ForwardDF**, we need to calculate the number of output links per document. 
Because we will need this information to calculate the PageRank we will collect this information in a Pandas Dataframe, and define a Broadcast variable.

In [0]:
def count_links(list_of_links):
  return list_of_links[0]

f = lambda 

In [0]:
udf_count_links=udf(count_links,LongType())

In [0]:
count_linksDF=ForwardDF.select("id",udf_count_links("links").alias("first_link")).show()

+-----+----------+
|   id|first_link|
+-----+----------+
|14800|     15166|
| 1359|     31260|
|23343|    311194|
|24026|      4231|
|34131|     14023|
|35430|     14533|
|25652|      3709|
|28413|     13960|
|10577|     13973|
|10622|     15614|
|11986|     15574|
|13049|     15783|
|30170|       658|
|44920|     15925|
|47128|     25392|
|50479|     14289|
|50847|     14960|
|50870|      4184|
|55578|     21635|
|60303|     13870|
+-----+----------+
only showing top 20 rows



In [0]:
print(type(ForwardDF.select("links")))

<class 'pyspark.sql.dataframe.DataFrame'>


In [0]:
# My approach with the size function instead of the len() one 
count_links2DF= ForwardDF.select("id",F.size("links").alias("count"))
# And the other one keeping the other row 
ForwardDF_WC= ForwardDF.select("id","links",F.size("links").alias("n_succesors"))

In [0]:
display(ForwardDF_WC.head(10))

id links n_succesors 14800 List(15165, 15414, 22936, 23340, 32616, 36916, 25445, 27619, 28436, 10172, 11887, 13621, 29618, 31459, 5646, 6591, 18010, 18710, 46268, 48768, 52041, 68282, 69692, 72971, 72987, 73213, 40157, 42207, 42927, 61309, 61645, 61768, 61829, 185441, 187749, 188848, 191436, 191747, 158364, 162300, 278508, 226651, 228690, 290921, 297040, 314629, 249524, 253026, 378212, 469097, 477189, 175143, 639333, 393363, 177922, 523499, 140639, 764177, 500205, 483397, 1095706, 1351766, 1360234, 613492, 96861, 1255782, 1379335, 1394879, 1609174, 621178, 1879959, 1758317, 885721, 2285933, 2999762, 1977720, 1978269, 2831787, 2134398, 2405818, 2407094, 106132, 3083341, 3828146, 4196468, 3328787, 3731174, 3534704, 5585097, 5151453, 7603861, 6170143, 658590, 8087628, 8103949, 8408318, 10035784, 7920576, 10518745, 10326216, 11126830, 9926348, 16861908, 19404136, 9762, 24660757, 14207476, 21468377, 21486360, 36756625, 16893191, 23564616, 21823121, 36756625, 22918149, 33064131, 300459, 303916, 14854602, 33064131, 16448388, 54938) 122 1359 List(31259, 5211, 5623, 200475, 266350, 467003, 467036, 635917, 140702, 606848, 889882, 2255715, 3046290, 4114697, 11440824, 37498649, 37498649, 37297795, 37297795, 7515928) 20 23343 List(311193, 397494, 354396, 123984, 1258722, 1387542, 1738560, 2307409, 1451666, 2580183, 2684862, 106597, 102352, 2379474, 4494157, 2727018, 7075178, 128497, 6797568, 8702589, 11254911, 23027978, 14207262, 37020363, 37020363, 39416992, 26766855, 39416992, 40877760, 23577486, 21076839, 24633607, 43047572, 40877760, 43047572, 40645144, 43970451, 40645144, 43970451, 29159039, 40164060, 40164060, 47541652, 47541652, 23707208) 45 24026 List(4230, 21505, 23634, 27119, 12582, 13311, 19836, 43427, 55172, 58894, 209459, 164901, 366847, 770467, 1075046, 1075071, 1075211, 969126, 2004504, 1983032, 9316, 20500703, 19739045, 34861007, 34861007) 25 34131 List(14022, 15040, 2844, 21175, 21655, 22830, 23635, 33685, 37400, 37401, 37410, 25508, 13017, 5180, 5636, 5914, 17514, 17955, 18728, 19325, 55285, 42896, 42909, 213639, 75220, 80441, 226375, 152735, 252814, 174645, 218052, 761886, 748731, 1064988, 1564830, 1214877, 2983189, 3213931, 106235, 3099755, 2360117, 4764461, 3508610, 4865795, 5997878, 8924479, 7505245, 7950369, 9668, 22778031, 12163656, 39472607, 39472607, 25416893, 20611504, 39042932, 28222445, 39042932, 12207509, 29926794, 29926794, 12246942, 47663449, 47663449, 15706527) 65 35430 List(14532, 15651, 16043, 2553, 2632, 25657, 13621, 28809, 30343, 168851, 442948, 8718425, 11523597, 8592, 13893101, 16685964, 46709309, 47612806, 47612806, 46709309) 20 25652 List(3708, 4146, 21244, 21482, 25645, 25649, 10597, 30118, 31816, 52593, 52710, 56689, 58893, 188171, 211583, 80750, 297696, 444649, 448570, 539055, 511846, 512047, 676659, 1094925, 1210483, 1863527, 1863625, 1863662, 2554181, 2558617, 2558635, 3220876, 1844863, 3042295, 3934892, 3339180, 3366673, 3468737, 30863462, 4582138, 30863462, 2121307, 662451, 9245142, 8500, 26429265, 26214389, 21490998, 42507334, 22558349, 22026933, 35576813, 35576813, 18630329, 31108048, 31108048, 18998750, 29067169, 29641327, 29641327, 2421503) 61 28413 List(13959, 14532, 1256, 1387, 21632, 21714, 34361, 37328, 25458, 25507, 25791, 25994, 26994, 10172, 10310, 13525, 29031, 29376, 30594, 31204, 31259, 6051, 20155, 20189, 45127, 45745, 47277, 47942, 48698, 49907, 73199, 197406, 199993, 200437, 159890, 159964, 325363, 204955, 163277, 75434, 75459, 75462, 148646, 148682, 148683, 148687, 291940, 299259, 368659, 361937, 468634, 175924, 437805, 632641, 383225, 393363, 217131, 219117, 770608, 538538, 899787, 845862, 1046450, 1105124, 1060851, 1138357, 1154819, 1404532, 1642481, 1003514, 1013789, 2894726, 1763184, 1738833, 2233278, 2315588, 2068003, 106286, 803136, 2513801, 3125814, 3828146, 4501200, 4543240, 4083381, 4070761, 6972565, 4590658, 4142501, 4151968, 4152108, 4657098, 6599943, 6645651, 2103585, 113295, 7778280, 8139097, 18949154, 7950310, 8261436, 10194547, 11650720, 30874803, 30874803, 2

**Note:** Instead use user defined function udf_count_links, check the pyspark.sql.functions library and use *size* function, which is more efficient!

In [0]:
display(count_linksDF)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-1782843817263998> in <module>
----> 1 display(count_linksDF)

NameError: name 'count_linksDF' is not defined

In [0]:
ForwardPDF_WC = ForwardDF_WC.toPandas()

In [0]:
ForwardPDF_WC[ForwardPDF_WC.id == 14800]

,id,links,n_succesors


# 5) Construct *Reverse Links Table* from *Forward Links Table*
Now, we will define the Reverse Links Table Dataframe (**ReverseDF**), transforming the **ForwardDF** to a Dataframe with [*id*,*list of ids*] or similar.

*Suggestion*: Maybe the *list of ids* could contains not only the id of the target document, also the number of output links. This will improve the Page Rank calcule.

In [0]:
def reverseId(id,links):
  if (len(links)>0):
    reverse = [ (tgt_id,id) for tgt_id in links ]
  else:
    reverse=[]
  return reverse

In [0]:
ForwardRDD = ForwardDF_WC.rdd

In [0]:
ReverseRDD=(ForwardRDD
 .flatMap(lambda r: reverseId(r.id,r.links))
 .groupByKey()
 .map(lambda r: (r[0],list(r[1])))
 )

In [0]:
reverseDF=spark.createDataFrame(ReverseRDD,["id","links"])

In [0]:
display(reverseDF.head(10))

id,links
22936,"List(14800, 755367, 695295, 99613, 2820179, 30842027, 8889165, 8335160, 7404524, 6913118, 30941589, 21206923, 21520483, 39423220, 21147386, 21157958, 21177114, 36280966, 14919799, 14926413, 14931741, 22132567, 20994598, 21002100, 21002445, 21016099, 21016803, 18473320, 21840417, 21840490, 21840798, 21840967, 20955131, 20961917, 20978621, 38935393, 28299316, 14314801, 21732964, 19830459, 19856012, 19857205, 36648500, 37863783, 36941482, 25512374, 46874490, 19899331, 19901610, 19917252, 21998533, 42047168, 45041066, 44537951, 21884721, 21884939, 21885285, 21885760, 21934691, 21667238, 21667950, 21668534, 21669593, 33572177, 30801143, 46306965, 33572543, 19684410)"
31108048,"List(25652, 25652)"
175924,List(28413)
25575836,List(28413)
3350364,List(10577)
10929004,List(10577)
266204,List(11986)
925736,"List(11986, 6416597)"
26840,"List(50479, 718098, 1720678, 3257073, 12790931, 35686750, 12265410, 35538163, 43622965, 33611377)"
572302,List(50479)


In [0]:
reverseDF_WC= reverseDF.select("id","links",F.size("links").alias("n_precessors")).withColumnRenamed("links","precessors").withColumnRenamed("id","id1")

In [0]:
display(reverseDF_WC.head(10))

id1,precessors,n_precessors
22936,"List(14800, 755367, 695295, 99613, 2820179, 30842027, 8889165, 8335160, 7404524, 6913118, 30941589, 21206923, 21520483, 39423220, 21147386, 21157958, 21177114, 36280966, 14919799, 14926413, 14931741, 22132567, 20994598, 21002100, 21002445, 21016099, 21016803, 18473320, 21840417, 21840490, 21840798, 21840967, 20955131, 20961917, 20978621, 38935393, 28299316, 14314801, 21732964, 19830459, 19856012, 19857205, 36648500, 37863783, 36941482, 25512374, 46874490, 19899331, 19901610, 19917252, 21998533, 42047168, 45041066, 44537951, 21884721, 21884939, 21885285, 21885760, 21934691, 21667238, 21667950, 21668534, 21669593, 33572177, 30801143, 46306965, 33572543, 19684410)",68
31108048,"List(25652, 25652)",2
175924,List(28413),1
25575836,List(28413),1
3350364,List(10577),1
10929004,List(10577),1
266204,List(11986),1
925736,"List(11986, 6416597)",2
26840,"List(50479, 718098, 1720678, 3257073, 12790931, 35686750, 12265410, 35538163, 43622965, 33611377)",10
572302,List(50479),1


In [0]:
joineddf = reverseDF_WC.join(ForwardDF_WC, ForwardDF_WC.id == reverseDF_WC.id1, how='left')
ReverseDF = joineddf.select("id1", "precessors", "n_precessors", "links", "n_succesors").withColumnRenamed("links","succesors").withColumnRenamed("id1","id")


In [0]:
ReverseDF.show()

+--------+--------------------+------------+---------+-----------+
|      id|          precessors|n_precessors|succesors|n_succesors|
+--------+--------------------+------------+---------+-----------+
|   22936|[14800, 755367, 6...|          68|     null|       null|
|31108048|      [25652, 25652]|           2|     null|       null|
|  175924|             [28413]|           1|     null|       null|
|25575836|             [28413]|           1|     null|       null|
| 3350364|             [10577]|           1|     null|       null|
|10929004|             [10577]|           1|     null|       null|
|  266204|             [11986]|           1|     null|       null|
|  925736|    [11986, 6416597]|           2|     null|       null|
|   26840|[50479, 718098, 1...|          10|     null|       null|
|  572302|             [50479]|           1|     null|       null|
|  291336|[55578, 1669998, ...|           3|     null|       null|
|   22326|[68554, 2751139, ...|           6|     null|       n

In [0]:
ReverseDF.filter("succesors is NOT NULL").show()

+--------+--------------------+------------+--------------------+-----------+
|      id|          precessors|n_precessors|           succesors|n_succesors|
+--------+--------------------+------------+--------------------+-----------+
|   67710|          [14932355]|           1|[27813, 63528, 28...|         15|
| 2313853|          [13149672]|           1|[14560, 14680, 21...|        198|
| 1340660|           [1340660]|           1|[1340660, 8215938...|          6|
|45503929|[45415907, 45415907]|           2|[1354, 170691, 10...|         11|
| 1339199| [5322775, 39993457]|           2|[10568, 66326, 53...|         59|
|   10622|  [113360, 33017686]|           2|[15613, 3132, 267...|         67|
|46971960|[46971960, 46971960]|           2|[155434, 153190, ...|         10|
|  367353|           [8517021]|           1|[2844, 22153, 339...|         79|
| 1163159|          [42343306]|           1|[14587, 3226, 338...|         54|
|17422222|            [103271]|           1|[3079, 322335, 33...

In [0]:
ReversePDF_WC =  reverseDF_WC.toPandas()

# 6) Initialize *Page Rank Table*
We define a Pandas DataFrame (**PageRankPDF**) with the ids of documents in the **ReverseDF**, with the initial value. 

This could be:

\\(\frac{0.85}{N}\\)

where *N* is the number of documents in the **ReverseDF**.

  File "<command-4378291230143460>", line 2
    return r = 0.2
             ^
SyntaxError: invalid syntax


In [0]:
#This is metacode
# Initialize the fisrt iteration for the ranks 
ReverseDF= ReverseDF.select("id", "precessors", "n_precessors","n_succesors").withColumn("rank", F.lit(0.2)).na.fill(0)


In [0]:
ReverseDF.show()

+--------+--------------------+------------+-----------+----+
|      id|          precessors|n_precessors|n_succesors|rank|
+--------+--------------------+------------+-----------+----+
|   22936|[14800, 755367, 6...|          68|          0| 0.2|
|31108048|      [25652, 25652]|           2|          0| 0.2|
|  175924|             [28413]|           1|          0| 0.2|
|25575836|             [28413]|           1|          0| 0.2|
| 3350364|             [10577]|           1|          0| 0.2|
|10929004|             [10577]|           1|          0| 0.2|
|  266204|             [11986]|           1|          0| 0.2|
|  925736|    [11986, 6416597]|           2|          0| 0.2|
|   26840|[50479, 718098, 1...|          10|          0| 0.2|
|  572302|             [50479]|           1|          0| 0.2|
|  291336|[55578, 1669998, ...|           3|          0| 0.2|
|   22326|[68554, 2751139, ...|           6|          0| 0.2|
|  204228|[37729, 256956, 4...|           3|          0| 0.2|
|   9943

In [0]:
ReversePDF= ReverseDF.toPandas()

In [0]:
display(ReversePDF.head(100))

id,precessors,n_precessors,n_succesors,rank
22936,"List(14800, 755367, 695295, 99613, 2820179, 30842027, 8889165, 8335160, 7404524, 6913118, 30941589, 21206923, 21520483, 39423220, 21147386, 21157958, 21177114, 36280966, 14919799, 14926413, 14931741, 22132567, 20994598, 21002100, 21002445, 21016099, 21016803, 18473320, 21840417, 21840490, 21840798, 21840967, 20955131, 20961917, 20978621, 38935393, 28299316, 14314801, 21732964, 19830459, 19856012, 19857205, 36648500, 37863783, 36941482, 25512374, 46874490, 19899331, 19901610, 19917252, 21998533, 42047168, 45041066, 44537951, 21884721, 21884939, 21885285, 21885760, 21934691, 21667238, 21667950, 21668534, 21669593, 33572177, 30801143, 46306965, 33572543, 19684410)",68,0,0.2
31108048,"List(25652, 25652)",2,0,0.2
175924,List(28413),1,0,0.2
25575836,List(28413),1,0,0.2
3350364,List(10577),1,0,0.2
10929004,List(10577),1,0,0.2
266204,List(11986),1,0,0.2
925736,"List(11986, 6416597)",2,0,0.2
26840,"List(50479, 718098, 1720678, 3257073, 12790931, 35686750, 12265410, 35538163, 43622965, 33611377)",10,0,0.2
572302,List(50479),1,0,0.2


In [0]:
PageRankPDF.size

Out[338]: 418500

In [0]:
PageRankPDF["precessors"]

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-1109003329934581> in <module>
----> 1 PageRankPDF["precessors"].toList()

/databricks/python/lib/python3.8/site-packages/pandas/core/generic.py in __getattr__(self, name)
   5463             if self._info_axis._can_hold_identifiers_and_holds_name(name):
   5464                 return self[name]
-> 5465             return object.__getattribute__(self, name)
   5466 
   5467     def __setattr__(self, name: str, value) -> None:

AttributeError: 'Series' object has no attribute 'toList'

In [0]:
mylist = PageRankPDF[PageRankPDF.id == 925736].iloc[0,1]
for k in mylist: 
  print(k)

11986
6416597


In [0]:
print(type(ReverseDF.precessors))

<class 'pyspark.sql.column.Column'>


In [0]:
if 

id,links,n_succesors
340927,"List(25391, 26779, 31769, 340917, 340934, 340947, 151210, 153223, 263163, 357847, 553623, 869878, 489179, 2481352, 3434750, 4980140, 3414021, 22309268)",18
2542615,"List(15237, 56357, 201611, 173547, 1134614, 5975550)",6
4254040,"List(128150, 10894321, 38383124, 38383124)",4
3509649,"List(17011, 21818, 36581, 12027, 38791, 81818, 3289662, 4321490, 5334607, 4699587, 36762240, 36762240, 41618052, 41618052)",14
4060126,"List(22512, 23014, 33116, 149349, 446206, 950398, 1439194, 2210466, 3283823, 4399928, 4014582)",11
7009414,"List(69080, 63403, 325569, 285314, 1168864, 6143251, 8455935, 1313445)",8
7159975,"List(192573, 192578, 167409, 7160158)",4
6667872,List(6667940),1
10919930,"List(414082, 528282, 3021828)",3
6572839,"List(30118, 2339717, 3097027, 27283349)",4


# 7) Recalculate *Page Rank Table* until:
 * All the *Page Rank* values are stable
 * Reach number of iterations (sugested value: 20 iterations)

Should define a PageRank function, who receives the *id* of the document, *list of links*, and current PageRank table (**PageRankPDF**)*, and returns the new PageRank of the document *id* 
The loop must use the **ReverseDF** as master of information.

Now, we define a loop:
* While the conditions of exit are false:
 * Calculate the new page ranks, invoking the PageRank function from the **ReverseDF**, creating a new Dataframe NewPageRankDF with the [id,new_pagerank] info. (Could contains more information, if you use the suggestion in section 5) )
 * Collect the new page ranks and compares it with the previous **PageRankPDF**, checking if the new Page Ranks vary more than a threshold. If not, the exit condition is complied.
 * Update the **PageRankPDF** with the new values,and update the UDF functions.

In [0]:
def new_pagerank(list_of_ids, pagerank):
    '''
    1 - Iterar por los diferentes ids de los preccessors 
    2 - Hacer el sumatorio ponderado (formula de PageRank) 
    '''
    
    new_page_rank = 0.0
    N= pagerank.shape[0]-1
        
    for k in list_of_ids: 
        temp= pagerank
        line = temp.query(f'id== {int(k)}', inplace = False)
        r = float(line['rank'])
        s = float(line['n_succesors'])
        new_page_rank += r/s 
        
    return float(new_page_rank)

In [0]:
def new_pagerank_nosuccessors(pagerank,size):
    '''
    Takes into account all the ids that do not have any succesor, and shares their rank with the whole dataframe  
    '''

    withNextDF = pagerank.filter(pagerank["n_succesors"]!=0)
    withoutNextDF = pagerank.filter(pagerank["n_succesors"]==0)
    
    temp= withoutNextDF.select("rank").rdd.reduce(add)
    sum= 0.0
    for i in temp: sum += i
    
    withNextDF2 = withNextDF.withColumn("rank",withNextDF.rank + F.lit(sum/(size-1)))
    UnionDF = withoutNextDF.unionByName(withNextDF2)
    
    return UnionDF

In [0]:
count= 0
temp= 2
# broadcast_PageRankPDF= sc.broadcast(ReversePDF)
PageRankPDF= ReversePDF
size = PageRankPDF.shape[0]
PrevRankPDF= PageRankPDF

while (count < 1) and (temp > 0.0001):
    conv= 0.0
    # First we take into account the contribution of the usual pages
    udf_new_pagerank = udf(lambda l: new_pagerank(l , PageRankPDF), FloatType())
    NewPageRankDF = ReverseDF.select("id",udf_new_pagerank("precessors").alias("rank"),"n_succesors")
    
    
    # Then the contribution of the non successor pages 
    NewPageRankDF_2 = new_pagerank_nosuccessors(NewPageRankDF, size)
    PageRankPDF = NewPageRankDF_2.toPandas() 
    
    temp = abs(PageRankPDF["rank"] - PrevRankPDF["rank"]).sum()/size
       
    PrevRankPDF = PageRankPDF
    count += 1 

FinalPageRankDF = NewPageRankDF_2.orderBy(F.desc("rank")).select("id", "rank")

---------------------------------------------------------------------------
PythonException                           Traceback (most recent call last)
<command-4377037375310960> in <module>
     14 
     15     # Then the contribution of the non successor pages
---> 16     NewPageRankDF_2 = new_pagerank_nosuccessors(NewPageRankDF, size)
     17     PageRankPDF = NewPageRankDF_2.toPandas()
     18 

<command-4377037375310959> in new_pagerank_nosuccessors(pagerank, size)
      7     withoutNextDF = pagerank.filter(pagerank["n_succesors"]==0)
      8 
----> 9     temp= withoutNextDF.select("rank").rdd.reduce(add)
     10     sum= 0.0
     11     for i in temp: sum += i

/databricks/spark/python/pyspark/rdd.py in reduce(self, f)
   1030             yield reduce(f, iterator, initial)
   1031 
-> 1032         vals = self.mapPartitions(func).collect()
   1033         if vals:
   1034             return reduce(f, vals)

/databricks/spark/python/pyspark/rdd.py in collect(self)
    965         

In [0]:
FinalPageRankDF.head(15).show()
count

---------------------------------------------------------------------------
PythonException                           Traceback (most recent call last)
<command-1109003329934587> in <module>
----> 1 NewPageRankDF.show()

/databricks/spark/python/pyspark/sql/dataframe.py in show(self, n, truncate, vertical)
    488         """
    489         if isinstance(truncate, bool) and truncate:
--> 490             print(self._jdf.showString(n, 20, vertical))
    491         else:
    492             print(self._jdf.showString(n, int(truncate), vertical))

/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    121                 # Hide where the exception came from that shows a non-